In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.api import ExponentialSmoothing    
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime

In [ ]:
# Load the data
df = pd.read_csv('data/Nat_Gas.csv')
df['Dates'] = pd.to_datetime(df['Dates'], format='%m/%d/%y')
df = df.sort_values('Dates')
df.set_index('Dates', inplace=True)
df.head()

In [ ]:
df.info()
df.describe()

In [ ]:
# Visualize data
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Prices'], marker='o')
plt.title('Natural Gas Prices (Monthly)')
plt.xlabel('Date')
plt.ylabel('Price')
plt.grid(True)
plt.show()


In [ ]:
# Decompose trend, seasonality, and residuals
decomposition = seasonal_decompose(df['Prices'], model='additive', period=12)
decomposition.plot()
plt.show()

In [ ]:
# Check for stationarity
result = adfuller(df['Prices'])
print('ADF Statistic:', result[0])
print('p-value:', result[1])

In [ ]:
# Comparing models
# Train-test split
train = df.iloc[:-12]
test = df.iloc[-12:]

# Holt-Winters (Exponential Smoothing) training on train set
model_hw = ExponentialSmoothing(train['Prices'], seasonal='add',  trend='add', seasonal_periods=12, freq='ME')
fit_hw = model_hw.fit()
pred_hw = fit_hw.forecast(12)

# SARIMAX training on train set
model_sarimax = SARIMAX(train['Prices'], order=(1, 1, 1), seasonal_order=(0, 1, 1, 12), freq='ME')
fit_sarimax = model_sarimax.fit(disp=False)
pred_sarimax = fit_sarimax.get_forecast(steps=12)
mean_pred = pred_sarimax.predicted_mean

plt.figure(figsize=(10, 5))
plt.plot(df.index, df['Prices'], label='Observed')
plt.plot(pred_hw, label='Holt-Winters Forecast', linestyle='--')
plt.plot(mean_pred, label='SARIMAX Forecast', linestyle='--')
plt.title('Natural Gas Price Forecast Comparison')
plt.legend()
plt.show()

# Evaluate models
mse_hw_pred = mean_squared_error(test['Prices'], pred_hw)
rmse_hw_pred = np.sqrt(mse_hw_pred)
mse_sarimax_pred = mean_squared_error(test['Prices'], mean_pred)
rmse_sarimax_pred = np.sqrt(mse_sarimax_pred)

print(f'Holt-Winters MSE: {mse_hw_pred}, RMSE: {rmse_hw_pred}')
print(f'SARIMAX MSE: {mse_sarimax_pred}, RMSE: {rmse_sarimax_pred}')

In [ ]:
# Extrapolate using Holt-Winters (Exponential Smoothing)
model_hw = ExponentialSmoothing(df['Prices'], seasonal='add',  trend='add', seasonal_periods=12, freq='ME')
fit_hw = model_hw.fit()
forecast_hw = fit_hw.forecast(12)

plt.figure(figsize=(10, 5))
plt.plot(df.index, df['Prices'], label='Observed')
plt.plot(forecast_hw, label='Holt-Winters Forecast', linestyle='--')
plt.title('Natural Gas Price Forecast Comparison')
plt.legend()
plt.show()

In [ ]:
# df_forecast = forecast_hw.reset_index()
# df_forecast.columns = ['Dates', 'Prices']
# df_forecast.to_csv('data/Nat_Gas_Forecast.csv', index=False)

In [ ]:
# Create daily interpolation
combined = pd.concat([df['Prices'], forecast_hw])
daily_index = pd.date_range(start=combined.index[0], end=combined.index[-1], freq='D')
daily = combined.reindex(combined.index.union(daily_index)).interpolate(method='cubic').reindex(daily_index)

def estimate_price(date):
    d = pd.to_datetime(date)
    return float(daily.loc[d])

# Example usage
print(estimate_price('2024-07-15'))